<a href="https://colab.research.google.com/github/amasnaoui/Classification_damaged_not_damaged_car/blob/main/90_VGG16classification_damage_car.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import cv2
from PIL import Image
import tensorflow as tf
from keras import backend as K
from keras.models import load_model
from keras.preprocessing.image import img_to_array
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow.keras.applications
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input

In [ ]:
base_dir = '/content/drive/MyDrive/new_cars_dataset/'
train_dir = '/content/drive/MyDrive/new_cars_dataset/training/'
train_damage_dir = '/content/drive/MyDrive/new_cars_dataset/training/00-damage/'
train_whole_dir = '/content/drive/MyDrive/new_cars_dataset/training/01-whole/'
test_dir = '/content/drive/MyDrive/new_cars_dataset/test/'
test_damage_dir = '/content/drive/MyDrive/new_cars_dataset/test/00-damage/'
test_whole_dir = '/content/drive/MyDrive/new_cars_dataset/test/01-whole/'
valid_dir = '/content/drive/MyDrive/new_cars_dataset/validation/'
valid_damage_dir = '/content/drive/MyDrive/new_cars_dataset/validation/00-damage/'
valid_whole_dir = '/content/drive/MyDrive/new_cars_dataset/validation/01-whole/'

In [ ]:
num_damage_train = len(os.listdir(train_damage_dir))
num_whole_train = len(os.listdir(train_whole_dir))
num_damage_validaition = len(os.listdir(valid_damage_dir))
num_whole_validation= len(os.listdir(valid_whole_dir))
num_damage_test = len(os.listdir(test_damage_dir))
num_whole_test= len(os.listdir(test_whole_dir))

In [ ]:
print("Total Training damage Images",num_damage_train)
print("Total Training whole Images",num_whole_train)
print("--")
print("Total validation damage Images",num_damage_validaition)
print("Total validation whole Images",num_whole_validation)
print("--")
print("Total Test damage Images", num_damage_test)
print("Total Test whole Images",num_whole_test)
total_train = num_damage_train+num_whole_train
total_validation = num_damage_validaition+num_whole_validation
total_test = num_damage_test+num_whole_test
print("Total Training Images",total_train)
print("--")
print("Total Validation Images",total_validation)
print("--")
print("Total Testing Images",total_test)

Total Training damage Images 2016
Total Training whole Images 2016
--
Total validation damage Images 320
Total validation whole Images 320
--
Total Test damage Images 110
Total Test whole Images 110
Total Training Images 4032
--
Total Validation Images 640
--
Total Testing Images 220


In [ ]:
IMG_SHAPE  = 224
batch_size = 32

In [ ]:
image_gen_train = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
train_data_gen = image_gen_train.flow_from_directory(batch_size = batch_size,
                                                     directory = train_dir,
                                                     shuffle= True,
                                                     target_size = (IMG_SHAPE,IMG_SHAPE),
                                                     class_mode = 'binary')

image_generator_validation = ImageDataGenerator(rescale=1./255.)

val_data_gen = image_generator_validation.flow_from_directory(batch_size=batch_size,
                                                              directory=valid_dir,
                                                              target_size=(IMG_SHAPE, IMG_SHAPE),
                                                              class_mode='binary')

image_gen_test = ImageDataGenerator(rescale=1./255.)

test_data_gen = image_gen_test.flow_from_directory(batch_size=batch_size,
                                                   directory=test_dir,
                                                   target_size=(IMG_SHAPE, IMG_SHAPE),
                                                   class_mode='binary')

Found 4032 images belonging to 2 classes.
Found 640 images belonging to 2 classes.
Found 220 images belonging to 2 classes.


In [ ]:
train_data_gen.class_indices

{'00-damage': 0, '01-whole': 1}

In [ ]:
pre_trained_model = VGG16(input_shape=(224, 224, 3), include_top=False, weights="imagenet")

58900480/58889256 [==============================] - 0s 0us/step


In [ ]:
for layer in pre_trained_model.layers:
  print(layer.name)
  layer.trainable = False

input_1
block1_conv1
block1_conv2
block1_pool
block2_conv1
block2_conv2
block2_pool
block3_conv1
block3_conv2
block3_conv3
block3_pool
block4_conv1
block4_conv2
block4_conv3
block4_pool
block5_conv1
block5_conv2
block5_conv3
block5_pool


In [ ]:
# last_layer = pre_trained_model.get_layer('block5_pool')
# last_output = last_layer.output
# x = tf.keras.layers.GlobalMaxPooling2D()(last_output)
# x = tf.keras.layers.Dense(512, activation='relu')(x)
# x = tf.keras.layers.Dropout(0.5)(x)
# x = tf.keras.layers.Dense(2, activation='sigmoid')(x)

In [ ]:
# useful for getting number of classes
from glob import glob
folders = glob('/content/drive/MyDrive/new_cars_dataset/training/*')

In [ ]:
from tensorflow.keras import Model
from tensorflow.keras import layers
# Flatten the output layer to 1 dimension
x = layers.Flatten()(pre_trained_model.output)
# Add fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(1024,activation='relu')(x)
# Add a DropOut rate of 0.2
x = layers.Dropout(0.2)(x)
# Add a final sigmoid layer for classification
# x = layers.Dense(2,activation='sigmoid')(x)
x = layers.Dense(2, activation='softmax')(x)

model = Model(pre_trained_model.input, x)

model.compile(optimizer=Adam(learning_rate=0.0001),
              loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
# model = tf.keras.Model(pre_trained_model.input, x)

In [ ]:
# model.compile(optimizer='adam', loss=tf.keras.losses.sparse_categorical_crossentropy, metrics=['acc'])

In [ ]:
# class myCallback(tf.keras.callbacks.Callback):
#   def on_epoch_end(self, epoch, logs={}) :
#     if (logs.get('acc')>0.959):
#       print("\nReached 99.9% accuracy so cancelling training!")
#       self.model.stop_training = True

In [ ]:
# callbacks = myCallback()
history = model.fit(
    train_data_gen,
    steps_per_epoch=(total_train//batch_size),
    epochs = 20,
    validation_data=val_data_gen,
    validation_steps=(total_validation//batch_size),
    batch_size = batch_size,
    verbose = 1
)

Epoch 1/20


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


126/126 [==============================] - 72s 566ms/step - loss: 0.4015 - accuracy: 0.8457 - val_loss: 0.2242 - val_accuracy: 0.9109
Epoch 2/20
126/126 [==============================] - 70s 552ms/step - loss: 0.2108 - accuracy: 0.9144 - val_loss: 0.1711 - val_accuracy: 0.9312
Epoch 3/20
126/126 [==============================] - 70s 558ms/step - loss: 0.1785 - accuracy: 0.9276 - val_loss: 0.2040 - val_accuracy: 0.9266
Epoch 4/20
126/126 [==============================] - 69s 547ms/step - loss: 0.1710 - accuracy: 0.9311 - val_loss: 0.1610 - val_accuracy: 0.9375
Epoch 5/20
126/126 [==============================] - 70s 558ms/step - loss: 0.1411 - accuracy: 0.9444 - val_loss: 0.1611 - val_accuracy: 0.9406
Epoch 6/20
126/126 [==============================] - 69s 548ms/step - loss: 0.1276 - accuracy: 0.9509 - val_loss: 0.1774 - val_accuracy: 0.9453
Epoch 7/20
 83/126 [==================>...........] - ETA: 21s - loss: 0.1366 - accuracy: 0.9443

In [ ]:
# vgg_classifier = model.fit(train_data_gen,
#                            steps_per_epoch=(total_train//batch_size),
#                            epochs = 20,
#                            validation_data=val_data_gen,
#                            validation_steps=(total_validation//batch_size),
#                            batch_size = batch_size,
#                            verbose = 1)

In [ ]:
result = model.evaluate(test_data_gen,batch_size=batch_size)
print("test_loss, test accuracy",result)

In [ ]:
from keras.preprocessing import image
import matplotlib.pyplot as plt
import numpy as np

dir_path = '/content/drive/MyDrive/test/'

for i in os.listdir(dir_path):
  img = image.load_img(dir_path + '//' + i, target_size=(224,224))
  plt.imshow(img)
  plt.show()

  X = image.img_to_array(img)
  X = np.expand_dims(X,axis=0)
  images = np.vstack([X])
  val = model.predict(images)
  if np.argmax(val) == 0 :
    print("damaged car")
  else :
    print("Not damaged car")